**This script does web scraping for user and app information based on user and app id, then prepare two dataframes for further processing.**

In [1]:
import requests, time
from bs4 import BeautifulSoup
import json 
import pandas as pd
import re
import os
import sys
os.getcwd()

'C:\\Users\\luziy\\Documents\\datacamp\\notes\\recommendation_system\\projects'

In [2]:
# a function to show work status
def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 1. * currentCount / totalCount * 100
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\rStatus: [{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print('\n')

### 1: user data aquisition and read

In [3]:
with open("..\\week1\\W1\\W1\\steam_user_id.txt" ,"r" ) as stmf:
   stmid__raw = [ line.rstrip('\n') for line in stmf.readlines()  ]
   stmid_lst = [x for x in stmid__raw  if len(x) != 0 ]

In [4]:
api_key = '2DDBE08C80748E49571B3E6708367FEC' 
url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
f = open(r'data/user_game_inventory.txt', "a+" ) 


total_count = len(stmid_lst)

current_count = 0

show_work_status( 1, total_count, current_count)

start_time = time.time()

# use the first 100 as illustration: 
for stmid in stmid_lst[0:10]:
    params = {'key': api_key , 'steamid' : stmid , 'format' : 'json'}
    r = requests.get(url, params)
    rj = r.json()   
    rl = rj.setdefault( 'response'  ).setdefault( 'games' )
    user_inventory = {}
    user_inventory['user_steam_id' ] = stmid
    
    if not rl:
        user_inventory['user_game_inventory'] = None

    else:
        user_inventory['user_game_inventory'] = rl
        
    f.write("%s\n"  % (json.dumps(user_inventory)) )
    
    show_work_status(1, total_count, current_count)
    
    current_count += 1
   
    time.sleep(10)
    
f.close

end_time = time.time()

running_time = end_time - start_time

Status: [                                                                                                    ] 0.20% 

In [8]:
with open("data\\user_game_inventory.txt" ,"r" ) as invt:
    ugg = [ line.rstrip('\n') for line in invt.readlines()]

In [9]:
steam_user_df = pd.DataFrame(  )

for raw_string in ugg[0:10]:
    js = json.loads(raw_string)
    if js.get('user_game_inventory', {} ) is not None:
        user_df = pd.DataFrame.from_dict(js.get('user_game_inventory', {} ))
        user_df.loc[ : , "user_id"] = js.get('user_steam_id', {} )
        user_df = user_df[  ["user_id" , "appid" , "playtime_forever"] ]
        steam_user_df = pd.concat( [steam_user_df , user_df], axis = 0 )

In [15]:
steam_user_df = pd.DataFrame(  )

for raw_string in ugg[0:10]:
    js = json.loads(raw_string)
    if js.get('user_game_inventory', {} ) is not None:
        user_df = pd.DataFrame.from_dict(js.get('user_game_inventory', {} ))
        user_df.loc[ : , "user_id"] = js.get('user_steam_id', {} )
        user_df = user_df[  ["user_id" , "appid" , "playtime_forever"] ]
        steam_user_df = pd.concat( [steam_user_df , user_df], axis = 0 )

In [16]:
steam_user_df.to_csv( r'data\user_df_clean.csv' , header = True )

### 2: app data aquisition and read

In [17]:
with open("C:/Users/luziy/Documents/datacamp/notes/recommendation_system/week1/W1/W1/app_id.txt" ,"r" ) as appf:
   appid__raw = [ line.rstrip('\n') for line in appf.readlines()  ]
   appid_lst = [x for x in appid__raw  if len(x) != 0 ]

In [18]:
# take the first 10 as demo
url = 'http://store.steampowered.com/api/appdetails/'
f = open(r'data/app_detail.txt', "a+" ) 
total_count = len(stmid_lst)
current_count = 161
show_work_status( 1, total_count, current_count)

start_time = time.time()

for appid in appid_lst[0:10]:
    params = [ ('appids', appid) ] 
    rr = requests.get(url, params)
    rrj = rr.json()   
    rrl = rrj.get(str(appid)).get('data')
    app_info = {}
    app_info[str(appid)] = rrl
        
    f.write("%s\n"  % ( json.dumps(app_info)) ) 

    show_work_status(1, total_count, current_count)
    
    current_count += 1
    
    time.sleep(5)
    
f.close

end_time = time.time()

running_time = end_time - start_time

Status: [>>>                                                                                                 ] 3.42% 

In [19]:
with open("data\\app_detail_dc.txt" ,"r" ) as invt:
    ugg = [ line.rstrip('\n') for line in invt.readlines()]

In [20]:
str_dict = { "name" : {}, "steam_appid" : {}, "release_date":{} ,   "currency":{}, "price": {}, "discount": {}, "is_free": {},  "required_age": {} ,  "language":{} ,
            "recommendations": {} , 
           "about_the_game":{} , "short_description":{} , "genres":{}, "windows_support": {}, "mac_support" : {}, "linux_support": {} ,
            "metacritic":{} ,  "achievements":{} ,  "url":{} , 
           "header_image":{} }

total_count = len(ugg)
current_count = 0
show_work_status( 1, total_count, current_count)
    
# discount , platforms

for raw_string in ugg:
    js = json.loads(raw_string)
    js1 = js.get( list(js.keys())[0], {} )
    if (js1 is not None) and ( bool(js1) ):   # not nonevalue  and not empty disc
        # first level dict
        steam_appid = js1.get('steam_appid', {} )
        name = js1.get('name', {} )
        steam_appid = js1.get('steam_appid', {} )
        required_age = js1.get('required_age', {} )
        is_free = js1.get('is_free', {} )
        about_the_game = js1.get('about_the_game', {} )
        short_description = js1.get('short_description', {} )
        supported_languages = js1.get('supported_languages', {} )
        header_image = js1.get('header_image', {} )
        language = js1.get("supported_languages", {})

        # second level dict
        currency = js1.get("price_overview", {}).get('currency')
        price = js1.get("price_overview", {}).get('final')
        discount = js1.get("price_overview", {}).get('discount_percent')

        metacritic = js1.get("metacritic", {}).get('score')
#        description = js1.get("genres", {})   
        if ( len(js1.get("genres", {})) >0 ):
            genres = list( map( lambda x : x.get( 'description' ) ,js1.get("genres", {}) )  )
        else:
            genres = {}
            
        recommendations = js1.get("recommendations", {}).get('total')
        achievements = js1.get("achievements", {}).get('total')
        release_date = js1.get("release_date", {}).get('date')
        url = js1.get("support_info", {}).get('url')
#        content_descriptors = js1.get("content_descriptors", {}).get('notes') 
        windows_support = js1.get("platforms", {}).get("windows")
        mac_support = js1.get("platforms", {}).get("mac")
        linux_support = js1.get("platforms", {}).get("linux")

        str_dict["currency"].update( {steam_appid: currency} )
        str_dict["price"].update( {steam_appid: price} )
        str_dict["discount"].update( {steam_appid: discount} )
        str_dict["language"].update( {steam_appid: language} )
        str_dict["metacritic"].update( {steam_appid: metacritic} )
        str_dict["recommendations"].update( {steam_appid: recommendations} )
        str_dict["achievements"].update( {steam_appid: achievements} )
        str_dict["release_date"].update( {steam_appid: release_date} )
        str_dict["url"].update( {steam_appid: url} )
#        str_dict["content_descriptors"].update( {steam_appid: content_descriptors} )
        str_dict["genres"].update( {steam_appid: genres} )    
        str_dict["short_description"].update( {steam_appid: short_description} )

        str_dict["name"].update( {steam_appid: name} )
        str_dict["steam_appid"].update( {steam_appid: steam_appid} )
        str_dict["required_age"].update( {steam_appid: required_age} )
        str_dict["is_free"].update( {steam_appid: is_free} )
        str_dict["about_the_game"].update( {steam_appid: about_the_game} )
        str_dict["header_image"].update( {steam_appid: header_image} )
        str_dict["windows_support"].update( {steam_appid: windows_support} )
        str_dict["mac_support"].update( {steam_appid: mac_support} )
        str_dict["linux_support"].update( {steam_appid: linux_support} )

        show_work_status(1, total_count, current_count)

        current_count += 1
    

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>        ] 92.43% 

In [21]:
app_df = pd.DataFrame.from_dict(str_dict)
app_df.to_csv( r'data\app_df.csv' , header = True )